In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
import json

from os.path import join
from pathlib import Path
from tqdm.notebook import tqdm
from itertools import combinations, combinations_with_replacement, product
from utils.style import rc_style
from utils.plotting import aligned_imshow_cbar
from utils.paralell import (agent_name, calculate_beta, isoelastic_utility, 
                            inverse_isoelastic_utility, wealth_change, 
                            shuffle_along_axis, create_gambles, 
                            create_gamble_pairs, create_trial_order, 
                            create_experiment, is_mixed, is_nobrainer, 
                            is_equal_growth, disagreement, bankruptcy_chance, 
                            richness_chance, experiment_duration, 
                            run_simulation, softmax, create_dmag_thrs, 
                            create_mag_thrs, create_var_thrs, 
                            is_g_deterministic)
mpl.rcParams.update(rc_style)                            

### Settings
- `x_0`: initial wealth
- `x_limit`: wealth limit per agent
- `n_fractals`: number of distinct fractals
- `n_simulations`: number of simulations run in paralell
- `eta_dynamic`: wealth dynamic parameter
- `eta_agent`: agent's risk attitude
- `c`: maximum fractal growth rate; should be adjusted for each dynamic
- `p_threshold`: critial choice probaiblity used to normalize softmax sensitivity across agents. Normalization ensures that all isoelastic agnets would choose the best fractal over the worst fractal with `p_threshold` probability. Values closer to one indicate more sensitive agents and more deterministic choices.
- `wealth_dependency`: decision if wealth dependency effects should be taken into account

In [ ]:
x_0 = 1000
x_limit = 4000
n_fractals = 9
n_trials = 360
p_threshold = 0.9999
agents = [-1, 0, 1]

# Gamble space settings 
eta_dynamic = 1
c_min = 0.001
c_max = 0.3
n_c = 100
min_unique_gambles = int(360 / 4)
max_mag_n = 7 

# Other
n_simulations = 1000
wealth_dependency = True

In [ ]:
n_mag_thrs = max_mag_n + 1
n_dmag_thrs = 2 * n_fractals - 2

# Store number of gambles after filtering
n_pairs = np.zeros((n_mag_thrs, n_mag_thrs, n_dmag_thrs), dtype=int)

mag_thrs = create_mag_thrs(1, n_fractals)
dmag_thrs = create_dmag_thrs(1, n_fractals)
for ml_idx, mh_idx in product(range(n_mag_thrs), repeat=2):
    for md_idx in range(n_dmag_thrs):
        
        ml_bound = mag_thrs[ml_idx]
        mh_bound = mag_thrs[-1 - mh_idx]
        md_bound = dmag_thrs[md_idx]

        gambles = create_gambles(1, n_fractals)
        gambles = [
            g for g in gambles 
            if np.mean(g) > ml_bound
            and np.mean(g) < mh_bound
            and not is_g_deterministic(g)
        ]
        
        gamble_pairs = create_gamble_pairs(gambles)
        gamble_pairs = [
            gp for gp in gamble_pairs
            if np.abs(np.mean(gp[0]) - np.mean(gp[1])) < md_bound
            if not is_nobrainer(gp)
        ]
        
        n_pairs[ml_idx, mh_idx, md_idx] = len(gamble_pairs)
    
    # remove redundant search values
    cutoff = np.where(np.diff(n_pairs[ml_idx, mh_idx]) == 0)[0]
    if len(cutoff):
        n_pairs[ml_idx, mh_idx, cutoff[0] + 1:] = 0
        cutoff_calc = 2 * n_fractals - 2 - ml_idx - mh_idx + int(ml_idx + mh_idx >= n_fractals - 1)
        
# remove max and min corners due to lower variance threshold
n_pairs[0, :] = 0
n_pairs[:, 0] = 0

# remove setups with not enough unique gamble pairs
n_pairs[n_pairs < min_unique_gambles] = 0

print("Gamble pair space subset: ", np.sum(n_pairs != 0))

In [ ]:
path_output = join("data", "v2", f"simulation_{agent_name(eta_dynamic)}")
Path(path_output).mkdir(exist_ok=False)

In [ ]:
n_agents = len(agents)

shape_common = (n_mag_thrs, n_mag_thrs, n_dmag_thrs, n_c)
shape_pair = (math.comb(n_agents, 2), ) + shape_common
shape_single = (n_agents, ) + shape_common

# Statistics
d_p = np.zeros(shape_pair)
d_y = np.zeros(shape_pair)
p_bank = np.zeros(shape_single)
p_rich = np.zeros(shape_single)
avg_len = np.zeros(shape_single)
n_gamble_pairs = np.zeros(shape_common)

c_range = np.linspace(c_min, c_max, n_c)
for c_idx, c in tqdm(list(enumerate(c_range))):
    
    mag_thrs = create_mag_thrs(c, n_fractals)
    dmag_thrs = create_dmag_thrs(c, n_fractals)
    
    for ml_idx, mh_idx in product(range(n_mag_thrs), repeat=2):
        for md_idx in range(n_dmag_thrs):
                        
            if n_pairs[ml_idx, mh_idx, md_idx] == 0:
                # Ignore excluded parameter values
                continue
        
            # Translate bound indices into average growth rates
            ml_bound = mag_thrs[ml_idx]
            mh_bound = mag_thrs[-1 - mh_idx]
            md_bound = dmag_thrs[md_idx]

            # Create & filter gambles 
            gambles = create_gambles(c, n_fractals=n_fractals)
            gambles = [
                g for g in gambles 
                if np.mean(g) > ml_bound
                and np.mean(g) < mh_bound
                and not is_g_deterministic(g)
            ]

            # Create & filter gamble pairs
            gamble_pairs = create_gamble_pairs(gambles)
            gamble_pairs = [
                gp for gp in gamble_pairs 
                if np.abs(np.mean(gp[0]) - np.mean(gp[1])) < md_bound
                if not is_nobrainer(gp)
            ]

            # Create experiment and randomized trial order
            experiment = create_experiment(gamble_pairs)
            trial_order = create_trial_order(
                n_simulations, 
                experiment.shape[-1], 
                n_trials
            )

            x, y, p = {}, {}, {}
            for agent in agents:

                # Estimate precision
                beta = calculate_beta(
                    eta_dynamic=eta_dynamic,
                    eta_agent=agent,
                    x_0=x_0,
                    x_limit=x_limit,
                    p_threshold=p_threshold,
                    c=c,
                )

                x[agent], p[agent], y[agent] = run_simulation(
                    experiment=experiment,
                    trial_order=trial_order,
                    eta_dynamic=eta_dynamic,
                    eta_agent=agent,
                    x_0=x_0,
                    x_limit=x_limit,
                    beta=beta,
                    wealth_dependency=wealth_dependency
                )

            # Store statistics
            for i, (a1, a2) in enumerate(combinations(agents, 2)):
                d_p[i, ml_idx, mh_idx, md_idx, c_idx] = disagreement(p[a1], p[a2])        
                d_y[i, ml_idx, mh_idx, md_idx, c_idx] = disagreement(y[a1], y[a2])
            for i, a in enumerate(agents):
                p_bank[i, ml_idx, mh_idx, md_idx, c_idx] = bankruptcy_chance(x[a])
                p_rich[i, ml_idx, mh_idx, md_idx, c_idx] = richness_chance(x[a], x_limit)
                avg_len[i, ml_idx, mh_idx, md_idx, c_idx] = experiment_duration(x[a], x_limit)

In [ ]:
# Store main data
np.save(join(path_output, "d_p.npy"), d_p)
np.save(join(path_output, "d_y.npy"), d_y)
np.save(join(path_output, "p_rich.npy"), p_rich)
np.save(join(path_output, "p_bank.npy"), p_bank)
np.save(join(path_output, "avg_len.npy"), avg_len)
np.save(join(path_output, "n_pairs.npy"), n_pairs)

# Create and store metadata
dimensions = {
    0: list(combinations(agents, 2)),
    1: list(create_mag_thrs(1, n_fractals)[:n_mag_thrs]),
    2: list(create_mag_thrs(1, n_fractals)[-n_mag_thrs:]),
    3: list(create_dmag_thrs(1, n_fractals)),
    4: list(c_range)
}

meta = {
    "x_0": x_0,
    "x_limit": x_limit,
    "n_fractals": n_fractals,
    "n_trials": n_trials,
    "p_threshold": p_threshold,
    "agents": agents,
    "eta_dynamic": eta_dynamic,
    "c_min": c_min,
    "c_max": c_max,
    "n_c": n_c,
    "min_unique_gambles": min_unique_gambles,
    "max_mag_n": max_mag_n,
    "n_simulations": n_simulations,
    "wealth_dependency": wealth_dependency,
    "dimensions": dimensions
}

with open(join(path_output, "metadata.json"), "w") as f:
    f.writelines(json.dumps(meta, indent=2))